In [22]:
import json
from api.post import post_raw

tools = [
    {
        "random_number_generator": {
            "name": "random_number_generator",
            "description": "Generates a random number x, s.t. range[0] <= x < range[1]",
            "params": [
                {
                    "name": "seed",
                    "description": "The random seed used by the generator",
                    "type": "int",
                    "required": True
                },
                {
                    "name": "range",
                    "description": "The range of the generated numbers",
                    "type": "tuple[int, int]",
                    "required": True
                }
            ]
        },
        "get_weather": {
            "name": "get_weather",
            "description": "Get the current weather for `city_name`",
            "params": [
                {
                    "name": "city_name",
                    "description": "The name of the city to be queried",
                    "type": "str",
                    "required": True
                }
            ]
        }
    }
]

system_info = {
    "role": "system",
    "content": "Answer the following questions as best as you can. You have access to the following tools: " + json.dumps(tools),
}

history = [system_info]
query = "北京的天气怎么样"
response = post_raw(query, history)
response_text = response['text'].split("<|observation|>")[0]

history.append({
    "role": "assistant",
    "content": response_text,
})

def parse_string(s):
    import re
    match = re.match(r"(\w+)\n ```python\n(\w+)\((\w+)='(\w+)'\)\n```", s)
    if match:
        name = match.group(1)
        function = match.group(2)
        key = match.group(3)
        value = match.group(4)
        return {'name': name, 'parameters': {key: value}}
    return None

function_call = parse_string(response_text)
print(function_call)

{'name': 'get_weather', 'parameters': {'city_name': '北京'}}


In [21]:
from tools.tool_register import get_tools, dispatch_tool
tools = get_tools()

tool_response = dispatch_tool(function_call['name'], function_call['parameters'])
print(tool_response)

{'current_condition': {'temp_C': '2', 'FeelsLikeC': '1', 'humidity': '25', 'weatherDesc': [{'value': 'Clear'}], 'observation_time': '10:41 AM'}}
